In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv( 'ml-1m/movies.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                    )
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users = pd.read_csv( 'ml-1m/users.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                    )
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings = pd.read_csv( 'ml-1m/ratings.dat',
                     sep = '::',
                     header = None,
                     engine = 'python',
                     encoding = 'latin-1'
                    )
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
training_set = pd.read_csv( 'ml-100k/u1.base', delimiter = '\t',  header=None )
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [6]:
training_set.shape

(80000, 4)

In [7]:
training_set = np.array( training_set, dtype = 'int' )
print(training_set)

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [8]:
test_set = pd.read_csv( 'ml-100k/u1.test', delimiter = '\t',  header=None )
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [9]:
test_set.shape

(20000, 4)

In [10]:
test_set = np.array( test_set, dtype = 'int' )
print(test_set)

[[        1         6         5 887431973]
 [        1        10         3 875693118]
 [        1        12         5 878542960]
 ...
 [      459       934         3 879563639]
 [      460        10         3 882912371]
 [      462       682         5 886365231]]


In [11]:
nb_users = int(max( max( training_set[ :, 0 ] ), max( test_set[ :, 0 ] ) ))
nb_movies = int(max( max( training_set[ :, 1 ] ), max( test_set[ :, 1 ] ) ))

print(nb_users)
print(nb_movies)

943
1682


In [12]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[ :, 1 ][data[ :, 0 ] == id_users]
        id_ratings = data[ :, 2 ][data[ :, 0 ] == id_users]
        
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        
        new_data.append( list(ratings) )
    
    return new_data

In [13]:
training_set = convert(training_set)
test_set = convert(test_set)

In [14]:
len(training_set)

943

In [15]:
len(test_set)

943

# Converting data into Torch tensors

In [16]:
training_set = torch.FloatTensor( training_set )
test_set = torch.FloatTensor( test_set )

In [17]:
type( training_set )

torch.Tensor

In [18]:
type( test_set )

torch.Tensor

# Creating the architechture of the Neural Network

In [19]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [20]:
sae = SAE()

In [21]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the SAE

In [22]:
nb_epoch = 200

for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion( output, target )
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    
    print( 'epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s) )

epoch: 1 loss: tensor(1.7709)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0382)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0266)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0219)
epoch: 9 loss: tensor(1.0210)
epoch: 10 loss: tensor(1.0198)
epoch: 11 loss: tensor(1.0192)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0176)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0170)
epoch: 17 loss: tensor(1.0169)
epoch: 18 loss: tensor(1.0167)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0162)
epoch: 22 loss: tensor(1.0161)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0160)
epoch: 25 loss: tensor(1.0157)
epoch: 26 loss: tensor(1.0157)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0152)
epoch: 29 loss: tensor(1.0130)
epoch: 30 loss: tensor(1.0118)
epoch: 31 loss: tensor(1.0089)
epoch: 32 loss: tensor(1.0083)
epoch: 33 loss: t

# Testing the SAE

In [24]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion( output, target )
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
print( "test loss: " + str( test_loss / s ) )

test loss: tensor(0.9559)
